Extractive Summarization Section

In [ ]:
import json
import pandas as pd
import numpy as np
from helper_functions import *

In [ ]:
clusters = pd.read_csv('./clustering_results.csv')

In [ ]:
df = pd.read_csv('./Telehealth.csv')

In [ ]:
clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  44 non-null     int64  
 1   Unnamed: 0.1                44 non-null     int64  
 2   year                        44 non-null     int64  
 3   class                       44 non-null     object 
 4   cleaned_pca_component_1     44 non-null     float64
 5   cleaned_pca_component_2     44 non-null     float64
 6   cleaned_pca_dbscan_class    44 non-null     int64  
 7   cleaned_tsne_dim_1          44 non-null     float64
 8   cleaned_tsne_dim_2          44 non-null     float64
 9   cleaned_tsne_dbscan_class   44 non-null     int64  
 10  lemma_pca_component_1       44 non-null     float64
 11  lemma_pca_component_2       44 non-null     float64
 12  lemma_pca_dbscan_class      44 non-null     int64  
 13  lemma_tsne_dim_1            44 non-nu

In [ ]:
df.head(1)

,Journal Title,Article Title,Date Published,Authors,Abstract,Keywords,Citation,Content,Content_Length,Abstract_Length,Parsed_Keywords,Parsed_Keywords_Length,Subfield,et_al_Count,Stopwords_Lemma_Longform_Clean_Content,Clean_Content,Classification
0,Psychological Services,The Effectiveness of Telepsychology With Veter...,2021,Michael J. McClellan; Richard Osbaldiston; Ron...,Veterans face a variety of stressors due to th...,"KEYWORDS:\n\ntelepsychology, meta-analysis, ve...","McClellan, M. J., Osbaldiston, R., Wu, R., Yea...",Veterans face a variety of stressors related t...,37477,2411,"['telepsychology,', 'meta-analysis,', 'veteran...",5,Clinical & Counseling Psychology,50,veteran face variety stressor relate military ...,veteran face variety stressor relate military ...,Covid


In [ ]:
merged = df.merge(clusters, on= 'Article Title')

In [ ]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 0 to 43
Data columns (total 45 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Journal Title_x                         44 non-null     object 
 1   Article Title                           44 non-null     object 
 2   Date Published                          44 non-null     int64  
 3   Authors                                 44 non-null     object 
 4   Abstract_x                              39 non-null     object 
 5   Keywords_x                              42 non-null     object 
 6   Citation                                44 non-null     object 
 7   Content                                 44 non-null     object 
 8   Content_Length_x                        44 non-null     int64  
 9   Abstract_Length                         44 non-null     int64  
 10  Parsed_Keywords                         44 non-null     object 


In [ ]:
# Drop everything we don't need to focus on the summaries:

summ_df = merged[['Abstract_x', 'Clean_Content', 'lemma_tsne_dbscan_class']].copy()

In [ ]:
# # Some of the articles do not include an Abstract, so we will drop those and redo the index:

# print(len(summ_df))
# summ_df.dropna(inplace=True)
# summ_df.reset_index(drop=True, inplace=True)
# print(len(summ_df))

44
39


That dropped 5 articles that did not have an abstract.(Not dropping articles without abstract) 

In [ ]:
# Create a new column with any references within parenthesis removed. I would imagine these could mess with any
# summarization algorithims, so we'll want to remove those (DBB):
# 
# summ_df['LF_no_refs'] = df['LongForm'].apply(remove_text_in_parens)

Create the three cluster corpora, exclude the three papers in the unassigned -1 category

In [ ]:
summ_df_0 = summ_df[summ_df['lemma_tsne_dbscan_class']==0]
summ_df_1 = summ_df[summ_df['lemma_tsne_dbscan_class']==1]
summ_df_2 = summ_df[summ_df['lemma_tsne_dbscan_class']==2]

In [ ]:
def create_text(df):
  text = ''
  abstracts = ''
  for i in df['Clean_Content']:
    text = text + i + ' '
  for j in df['Abstract_x']:
    try:
      abstracts = abstracts + j + ' '
    except:
      continue
  return text, abstracts

Corpus0, abstracts0 = create_text(summ_df_0)
Corpus1, abstracts1 = create_text(summ_df_1)
Corpus2, abstracts2 = create_text(summ_df_2)
Corpus0

"veteran face variety stressor relate military service Institute Medicine 2014 Tanielian  2008 veteran return military conflict mental health condition veteran experience period readjustment reintegrate life family friend community substance Abuse Mental Health Services Administration SAMHSA 2012 p. 1 Tanielian  2008 individual juggle variety conflict family military relate responsibility learn readjust environment behavior associate constant state readiness long need process combat relate exposure collectively contribute problem veteran family Institute Medicine 2014 SAMHSA 2012).veterans expose combat likely report normal reaction stress include depression hopelessness insomnia nightmare feeling rejection aggressive behavior substance misuse SAMHSA 2012 report high level posttraumatic stress disorder posttraumatic stress disorder major depression generalized anxiety intermittent explosive disorder substance misuse return conflict Lazar 2014 SAMHSA 2012 Tanielian  2008 veteran posttra

<h1>GENSIM Summary Section:

In [ ]:
from gensim.summarization import summarize

In [ ]:
# Dr. Diana said this was a good ratio to use for summaries:
summ_ratio = 0.05

In [ ]:
corpora = [Corpus0, Corpus1, Corpus2]
new_table = []
for i,text in enumerate(corpora):
  print(summarize(text, ratio=summ_ratio))
  new_table.append(summarize(text,ratio=summ_ratio))
Summaries = pd.DataFrame(new_table, columns=['Summaries'])
Summaries['Abstracts'] = [abstracts0, abstracts1, abstracts2]

356 accessibility increasingly important consider approximately 20 jail inmate United States house rural jail difficult reach site treatment Providers Kang Brown Subramanian 2017).there sound reasoning forensic e mental health good option evaluation intervention justice involve individual individual incarcerate high level mental health need general public Bureau Justice Statistics 2017 arguably need psychological attention frequently diagnose co occurring mental health substance use disorder high rate traumatic brain injury require complex treatment approach Beaudette Stewart 2016 Bronson Berzofsky 2017 Kessler  2003 Prins 2014 Shiroma  2012 Sung  2010 vivo psychiatric e mental health modality medication management critical frequently insufficient form mental health intervention psychotherapy cope skill development system place address need justice involve individual underresourced marginalize forensic setting medication management extent mental health treatment available coronavirus d

In [ ]:
Summaries

,Summaries,Abstracts
0,356 accessibility increasingly important consi...,Veterans face a variety of stressors due to th...
1,anxiety depressive disorder common impair ment...,Many university training clinics are facing nu...
2,462 copyright 2017 American Psychological Asso...,Although the medical impacts of COVID-19 are n...


In [ ]:
# def gnsm_summary(text):
#     summary = summarize(text, ratio=summ_ratio)
#     return summary

In [ ]:
# Create a column with a gensim summary for each paper:

# summ_df['gnsm_summ'] = summ_df['LF_no_refs'].apply(gnsm_summary)

In [ ]:
# summ_df.head()

,Abstract,LongForm,LF_no_refs,gnsm_summ
0,Veterans face a variety of stressors due to th...,Veterans face a variety of stressors related t...,Veterans face a variety of stressors related t...,Telepsychology has generally been found to be ...
1,The COVID-19 pandemic and its requirements for...,The impact of the coronavirus disease 2019 pan...,The impact of the coronavirus disease 2019 pan...,"Finally, because the stakes are so significant..."
2,Forensic e-mental health is an area of psychol...,"In November 2019, coronavirus disease 2019—the...","In November 2019, coronavirus disease 2019—the...","Accessibility is increasingly important, consi..."
3,Many university training clinics are facing nu...,Anxiety and depressive disorders are among the...,Anxiety and depressive disorders are among the...,A review of telepsychology’s specific ethical ...
4,The emergence of the Covid-19 pandemic at the ...,"In mid-March, 2020, the authors—as well as the...","In mid-March, 2020, the authors—as well as the...",The uncertain duration of this new professiona...


<h1>BLEU Scoring Section:

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [ ]:
# bleu_score_dict = {}

# for row in range(len(summ_df)):
#     # Using (1,0,0,0) weights to only search for unigram similarities since beyond that the scores were infintesimal
    
#     bleu_score = sentence_bleu(summ_df.Abstract[row], summ_df.gnsm_summ[row], weights=(1,0,0,0))
#     print(bleu_score)
#     bleu_score_dict[row] = bleu_score

In [ ]:
bleu_score_dict = {}

for row in range(len(Summaries)):
    # Using (1,0,0,0) weights to only search for unigram similarities since beyond that the scores were infintesimal
    
    bleu_score = sentence_bleu(Summaries.Abstracts[row], Summaries.Summaries[row])
    print(bleu_score)
    bleu_score_dict[row] = bleu_score
Summaries['Bleu Score'] = Summaries.from_dict(bleu_score_dict.values())

In [ ]:
# Create a column for the bleu score from the blue_score_dict:
summ_df['gnsm_bleu'] = summ_df.from_dict(bleu_score_dict.values())

In [ ]:
summ_df.head()

,Abstract,LongForm,LF_no_refs,gnsm_summ,gnsm_bleu
0,Veterans face a variety of stressors due to th...,Veterans face a variety of stressors related t...,Veterans face a variety of stressors related t...,Telepsychology has generally been found to be ...,0.018680
1,The COVID-19 pandemic and its requirements for...,The impact of the coronavirus disease 2019 pan...,The impact of the coronavirus disease 2019 pan...,"Finally, because the stakes are so significant...",0.005193
2,Forensic e-mental health is an area of psychol...,"In November 2019, coronavirus disease 2019—the...","In November 2019, coronavirus disease 2019—the...","Accessibility is increasingly important, consi...",0.007988
3,Many university training clinics are facing nu...,Anxiety and depressive disorders are among the...,Anxiety and depressive disorders are among the...,A review of telepsychology’s specific ethical ...,0.017363
4,The emergence of the Covid-19 pandemic at the ...,"In mid-March, 2020, the authors—as well as the...","In mid-March, 2020, the authors—as well as the...",The uncertain duration of this new professiona...,0.010333


<h1>Rouge Scoring Section:

In [ ]:
from rouge import rouge_score

rouge_scorer = rouge_score.rouge_n

In [ ]:
rouge_scores_dict = {}

for row in range(len(summ_df)):
    
    rouge_scores = rouge_scorer(summ_df.Abstract[row], summ_df.gnsm_summ[row])
    print(rouge_scores)
    rouge_scores_dict[row] = rouge_scores

{'f': 0.7098515469575528, 'p': 0.7185792349726776, 'r': 0.7013333333333334}
{'f': 0.6514745261692387, 'p': 0.8804347826086957, 'r': 0.5170212765957447}
{'f': 0.7154046949462809, 'p': 0.898360655737705, 'r': 0.5943600867678959}
{'f': 0.7471074330742435, 'p': 0.837037037037037, 'r': 0.6746268656716418}
{'f': 0.6833114275227458, 'p': 0.8524590163934426, 'r': 0.5701754385964912}
{'f': 0.7361299003050973, 'p': 0.7953216374269005, 'r': 0.6851385390428212}
{'f': 0.6814268094019539, 'p': 0.8147058823529412, 'r': 0.5856236786469344}
{'f': 0.7483660080721091, 'p': 0.7435064935064936, 'r': 0.7532894736842105}
{'f': 0.7072135735569743, 'p': 0.7911392405063291, 'r': 0.639386189258312}
{'f': 0.7399267349912115, 'p': 0.6644736842105263, 'r': 0.8347107438016529}
{'f': 0.7176079684462645, 'p': 0.7714285714285715, 'r': 0.6708074534161491}
{'f': 0.7258805464849007, 'p': 0.8977272727272727, 'r': 0.609254498714653}
{'f': 0.7170953052149931, 'p': 0.8200692041522492, 'r': 0.6370967741935484}
{'f': 0.72282608

In [ ]:
# Isolate the F1 scores:

rouge_f1_dict = {}
row = 0

for score_dict in rouge_scores_dict.values():
    f1_score = score_dict['f']
    print(f1_score)
    rouge_f1_dict[row] = f1_score
    row += 1

0.7098515469575528
0.6514745261692387
0.7154046949462809
0.7471074330742435
0.6833114275227458
0.7361299003050973
0.6814268094019539
0.7483660080721091
0.7072135735569743
0.7399267349912115
0.7176079684462645
0.7258805464849007
0.7170953052149931
0.7228260820641836
0.7152103509875264
0.7453987681014059
0.7357664184842667
0.6844319725832222
0.729577459887919
0.7067448630792219
0.7745664689934931
0.6848874548668336
0.7715735990746706
0.6791044726144465
0.6731571577392492
0.6810810760971879
0.5351473873377862
0.7030129076181791
0.6550632861490547
0.6372360797138238
0.680921047718144
0.6878306829327288
0.6277873020333108
0.6269841221164809
0.6470588188346021
0.6832116739881294
0.6948356757514799
0.6996805061833845
0.6648575255461928


In [ ]:
# Create a column for the rouge F1 scores from the rouge_scores_dict:
summ_df['gnsm_rouge_f1'] = summ_df.from_dict(rouge_f1_dict.values())

In [ ]:
summ_df.head()

,Abstract,LongForm,LF_no_refs,gnsm_summ,gnsm_bleu,gnsm_rouge_f1
0,Veterans face a variety of stressors due to th...,Veterans face a variety of stressors related t...,Veterans face a variety of stressors related t...,Telepsychology has generally been found to be ...,0.018680,0.709852
1,The COVID-19 pandemic and its requirements for...,The impact of the coronavirus disease 2019 pan...,The impact of the coronavirus disease 2019 pan...,"Finally, because the stakes are so significant...",0.005193,0.651475
2,Forensic e-mental health is an area of psychol...,"In November 2019, coronavirus disease 2019—the...","In November 2019, coronavirus disease 2019—the...","Accessibility is increasingly important, consi...",0.007988,0.715405
3,Many university training clinics are facing nu...,Anxiety and depressive disorders are among the...,Anxiety and depressive disorders are among the...,A review of telepsychology’s specific ethical ...,0.017363,0.747107
4,The emergence of the Covid-19 pandemic at the ...,"In mid-March, 2020, the authors—as well as the...","In mid-March, 2020, the authors—as well as the...",The uncertain duration of this new professiona...,0.010333,0.683311


<h1>Keyword Extraction Section:

In [ ]:
from rake_nltk import Rake

In [ ]:
df_overview = pd.read_csv("./Data/processed/LongForm_Clean_Lemma_Telehealth.csv")

In [ ]:
from rake_nltk import Rake

keyword_corpus = " ".join(df_overview["LongForm_Clean_Content_Lemma"].str.replace("et al",""))

r_extraction = Rake()

r_extraction.extract_keywords_from_text(keyword_corpus)


r_extraction.get_ranked_phrases()[:10]

['treat various disorder bee 2008 postel de haan de jong 2008 include posttraumatic stress disorder germain marchand bouchard drouin guay 2009 depression sloan gallagher feinstein lee pruneau 2011 anxiety ruskin 2004 substance use frueh henderson myrick 2005 chronic pain macea gajos calil fregni 2010',
 'largely positive adler pritchett kauth nadorff 2014 baird whitney caedo 2018 brooks manson bair dailey shore 2012 cunningham connors lever stephan 2013 levy strachan 2013 mitchell maclaren morton carachi 2009 moreau 2018 whitten kuwahara 2004 wynn bergvik pettersen fossum 2012',
 'neuropsychological testing bouchard 2004 cullum hynan grosch parikh weiner 2014 cullum weiner gehrmann hynan 2006 gehrman shah miles kuna godleski 2016 gros yoder tuerk lozano acierno 2011 hilty 2013 morland 2014 morland hynes mackintosh resick chard 2011the veteran ’',
 'encopresis eg davis sampilo gallagher landrum malone 2013 palermo wilson peters lewandowski somhegyi 2009 richardson frueh grubaugh egede e